In [112]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [113]:
response = requests.get('https://www.epicurious.com/search/cucumbers')
doc = BeautifulSoup(response.text)

In [114]:
rows =[]
url = ''
for page_num in range(1,11): 
    url = f'https://www.epicurious.com/search/cucumbers?{page_num}'
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    cucumbers = doc.find_all('article')
    for cucumber in cucumbers:
        row = {}
        row['tag'] = cucumber['class'][0]
        a_link = cucumber.find('a', class_='view-complete-item',itemprop = 'url')
        try:
            row['title'] = a_link['title']
        except:
            pass
        try:
            row['summary'] = cucumber.find('header',class_= 'summary').find('p',class_='dek').text
        except:
            pass
        try:
            row['rating'] = cucumber.find('span',itemprop = 'ratingValue').text
        except:
            pass
        try:
            row['would make again'] = cucumber.find('dd',class_ ='make-again-percentage').text
        except:
            pass
        try:
            row['url'] = f"https://www.epicurious.com{a_link['href']}"
        except:
            pass
        rows.append(row)
      



In [116]:
df = pd.DataFrame(rows)
df

,rating,summary,tag,title,url,would make again
0,4,These quick pickles have just the right amount...,recipe-content-card,Spicy Lightly Pickled Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%
1,4,Think of this as an all-purpose garlic sauce. ...,recipe-content-card,Cucumbers with Ajo Blanco Sauce,https://www.epicurious.com/recipes/food/views/...,100%
2,2,Beef tenderloin is precious enough to baby on ...,recipe-content-card,Cold Beef Tenderloin with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,0%
3,3,Take your cast-iron pan to the grill for this ...,recipe-content-card,Fried Fish Sandwiches with Cucumbers and Tarta...,https://www.epicurious.com/recipes/food/views/...,100%
4,3,We call for semi-pearled grains because they c...,recipe-content-card,Grain Salad with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%
5,2,Go hard when charring the cabbage wedges—to th...,recipe-content-card,Charred Cabbage with Goat Cheese Raita and Cuc...,https://www.epicurious.com/recipes/food/views/...,0%
6,0,A ginger and coconut aminos dressing brings to...,recipe-content-card,Cucumber-Seaweed Salad,https://www.epicurious.com/recipes/food/views/...,0%
7,3.5,Quickly caramelized pineapple and sautéed shri...,recipe-content-card,Pineapple Shrimp Noodle Bowls,https://www.epicurious.com/recipes/food/views/...,73%
8,3.5,"The shrimp cooks in minutes, so while it’s mar...",recipe-content-card,Sambal Shrimp Lettuce Wraps,https://www.epicurious.com/recipes/food/views/...,50%
9,0,"If you have ever peeled, seeded, and sliced cu...",recipe-content-card,Smashed Cucumber Salad with Lemon and Celery Salt,https://www.epicurious.com/recipes/food/views/...,0%


In [117]:
df.to_csv("epicurious_cucumbers.csv", index=False)

### Part 2

In [118]:
df = pd.read_csv("epicurious_cucumbers.csv")
df = df[df['tag']=='recipe-content-card']


In [130]:
def scrape_page(row):
    url = row['url']
    print("Scraping", url)
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    page = {}
    steps = doc.find_all('li', class_="preparation-step")
    page['directions'] = '\n'.join([step.text for step in steps])
    ingredients = doc.find_all('li', class_="ingredient")
    page['ingredients'] = '\n'.join([ingredient.text for ingredient in ingredients])
    tags = doc.find_all('dt', itemprop="recipeCategory")
    page['tags'] = '\n'.join([tag.text for tag in tags])
    return pd.Series(page)

In [136]:
scraped_df = df.apply(scrape_page, axis=1)
scraped_df

Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumbers-with-ajo-blanco-sauce
Scraping https://www.epicurious.com/recipes/food/views/cold-beef-tenderloin-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/fried-fish-sandwiches-with-cucumbers-and-tartar-sauce
Scraping https://www.epicurious.com/recipes/food/views/farro-spelt-grain-salad-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/grilled-charred-cabbage-with-goat-cheese-raita-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumber-seaweed-salad
Scraping https://www.epicurious.com/recipes/food/views/pineapple-shrimp-noodle-bowls
Scraping https://www.epicurious.com/recipes/food/views/sambal-shrimp-lettuce-wraps
Scraping https://www.epicurious.com/recipes/food/views/sm

Scraping https://www.epicurious.com/recipes/food/views/farro-spelt-grain-salad-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/grilled-charred-cabbage-with-goat-cheese-raita-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumber-seaweed-salad
Scraping https://www.epicurious.com/recipes/food/views/pineapple-shrimp-noodle-bowls
Scraping https://www.epicurious.com/recipes/food/views/sambal-shrimp-lettuce-wraps
Scraping https://www.epicurious.com/recipes/food/views/smashed-cucumber-salad-with-lemon-and-celery-salt
Scraping https://www.epicurious.com/recipes/food/views/thai-style-squid-and-cucumber-salad
Scraping https://www.epicurious.com/recipes/food/views/spicy-grilled-chicken-with-crunchy-fennel-salad
Scraping https://www.epicurious.com/recipes/food/views/grilled-salmon-with-meyer-lemons-and-creamy-cucumber-salad
Scraping https://www.epicurious.com/recipes/food/views/smashed-cucumber-salad-with-hot-vinegar
Scraping https://www.ep

,directions,ingredients,tags
0,Toss c...,"2 lb. medium Persian cucumbers (about 12), cut...",Bon Appétit\nPickles\nHors D'Oeuvre\nAppetizer...
1,Prehea...,5 mini seedless or Persian cucumbers (about 12...,Bon Appétit\nSauce\nCondiment\nGarlic\nAlmond\...
2,Prepar...,"1/4 cup extra-virgin olive oil, plus more\n1 (...",Bon Appétit\nDinner\nBeef Tenderloin\nBeef\nTo...
3,Mix ma...,1/2 cup mayonnaise\n1/4 cup finely grated lemo...,Bon Appétit\nSandwich\nFish\nSeafood\nFry\nBee...
4,Prehea...,2 cups semi-pearled farro or spelt\nKosher sal...,Bon Appétit\nSalad\nSide\nGrains\nCucumber\nTo...
5,Pulse ...,"1 garlic clove, smashed\n4 oz. fresh goat chee...",Bon Appétit\nCabbage\nVegetarian\nGrill/Barbec...
6,In a b...,"3/4 ounce dried wakame seaweed, cut into 1-inc...",HarperCollins\nSalad\nSide\nCucumber\nGinger\n...
7,Cook r...,12 oz. pad Thai–style rice noodles\n1 lb. larg...,Shrimp\nNoodle\nPineapple\nSoy Sauce\nGinger\n...
8,Whisk ...,1/2 cup hot chili paste (such as sambal oelek)...,Bon Appétit\nDinner\nSeafood\nShellfish\nShrim...
9,Gently...,5 mini seedless or Persian cucumbers or 1 smal...,Bon Appétit\nSalad\nCucumber\nSide\nLemon\nQui...


In [137]:
df.merge(scraped_df, left_index=True, right_index=True)

,rating,summary,tag,title,url,would make again,directions,ingredients,tags
0,4.0,These quick pickles have just the right amount...,recipe-content-card,Spicy Lightly Pickled Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%,Toss c...,"2 lb. medium Persian cucumbers (about 12), cut...",Bon Appétit\nPickles\nHors D'Oeuvre\nAppetizer...
1,4.0,Think of this as an all-purpose garlic sauce. ...,recipe-content-card,Cucumbers with Ajo Blanco Sauce,https://www.epicurious.com/recipes/food/views/...,100%,Prehea...,5 mini seedless or Persian cucumbers (about 12...,Bon Appétit\nSauce\nCondiment\nGarlic\nAlmond\...
2,2.0,Beef tenderloin is precious enough to baby on ...,recipe-content-card,Cold Beef Tenderloin with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,0%,Prepar...,"1/4 cup extra-virgin olive oil, plus more\n1 (...",Bon Appétit\nDinner\nBeef Tenderloin\nBeef\nTo...
3,3.0,Take your cast-iron pan to the grill for this ...,recipe-content-card,Fried Fish Sandwiches with Cucumbers and Tarta...,https://www.epicurious.com/recipes/food/views/...,100%,Mix ma...,1/2 cup mayonnaise\n1/4 cup finely grated lemo...,Bon Appétit\nSandwich\nFish\nSeafood\nFry\nBee...
4,3.0,We call for semi-pearled grains because they c...,recipe-content-card,Grain Salad with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%,Prehea...,2 cups semi-pearled farro or spelt\nKosher sal...,Bon Appétit\nSalad\nSide\nGrains\nCucumber\nTo...
5,2.0,Go hard when charring the cabbage wedges—to th...,recipe-content-card,Charred Cabbage with Goat Cheese Raita and Cuc...,https://www.epicurious.com/recipes/food/views/...,0%,Pulse ...,"1 garlic clove, smashed\n4 oz. fresh goat chee...",Bon Appétit\nCabbage\nVegetarian\nGrill/Barbec...
6,0.0,A ginger and coconut aminos dressing brings to...,recipe-content-card,Cucumber-Seaweed Salad,https://www.epicurious.com/recipes/food/views/...,0%,In a b...,"3/4 ounce dried wakame seaweed, cut into 1-inc...",HarperCollins\nSalad\nSide\nCucumber\nGinger\n...
7,3.5,Quickly caramelized pineapple and sautéed shri...,recipe-content-card,Pineapple Shrimp Noodle Bowls,https://www.epicurious.com/recipes/food/views/...,73%,Cook r...,12 oz. pad Thai–style rice noodles\n1 lb. larg...,Shrimp\nNoodle\nPineapple\nSoy Sauce\nGinger\n...
8,3.5,"The shrimp cooks in minutes, so while it’s mar...",recipe-content-card,Sambal Shrimp Lettuce Wraps,https://www.epicurious.com/recipes/food/views/...,50%,Whisk ...,1/2 cup hot chili paste (such as sambal oelek)...,Bon Appétit\nDinner\nSeafood\nShellfish\nShrim...
9,0.0,"If you have ever peeled, seeded, and sliced cu...",recipe-content-card,Smashed Cucumber Salad with Lemon and Celery Salt,https://www.epicurious.com/recipes/food/views/...,0%,Gently...,5 mini seedless or Persian cucumbers or 1 smal...,Bon Appétit\nSalad\nCucumber\nSide\nLemon\nQui...


In [139]:
df.to_csv("epicurious_cucumber_recipes.csv", index=False)